In [2]:
import pandas as pd
marks_excel = pd.read_excel('original.xlsx')
sub_excel = pd.read_excel('sublist.xlsx')

import numpy as np


In [3]:
# create marks and sub data
marks = marks_excel.copy()
sub_data = sub_excel.copy()
sub_data = sub_data.T
sub_data.columns = ['Subject Code','SUBJECTS','CREDIT HOUR(CH)']


In [4]:
#access by last digit of registeration no in marks 
def student(last_digit):
    regno = 782330220000 + last_digit
    return marks.loc[marks['rn'] == regno]

In [57]:
# select only sub th and sub in marks
def marksheet(last_digit):
    temp = student(last_digit)
    temp = temp.iloc[:,4:] # remove details other than marks
    temp = temp.drop(['en','ne','so','ec','bu','ac','ch','ed','on','oh'],axis=1)
    temp = temp.dropna(axis = 1) # drop unselected and subjects that has no marks
    return temp
# select student regno, roll no, name , dob
def student_info(last_digit):
    temp = student(last_digit)
    temp = temp[['rn','sn','name','dob']]
    temp.columns = ['REGISTERATON NO','SYMBOL NO','NAME','DOB']
    return temp

In [58]:
# helper functions 
def re_scale(num): # scale number in 100 scale
    #caution assumes no theory number assigned to students is less than or equall to 25
    if (num > 25):
        scaled_mark = (num/75)*100
    else:
        scaled_mark = (num/25)*100
    return scaled_mark

def conv_into_gp(num): # convert scaled number into gp
    if (num >= 90):
        gp = 4.0
    elif (num >= 80 and num < 90):
        gp = 3.6
    elif (num >= 70 and num < 80):
        gp = 3.2
    elif (num >= 60 and num < 70):
        gp = 2.8
    elif (num >= 50 and num < 60):
        gp = 2.4
    elif (num >= 40 and num < 50):
        gp = 2.0
    elif (num >= 35 and num < 40):
        gp = 1.6
    elif (num < 35):
        gp = 0
    return gp

def gp_to_letter(num):
    if (num == 4.0):
        l = 'A+'
    elif(num == 3.6):
        l = 'A'
    elif(num == 3.2):
        l = 'B+'
    elif (num == 2.8):
        l = 'B'
    elif (num == 2.4):
        l = 'C+'
    elif (num == 2.0):
        l = 'C'
    elif(num == 1.6):
        l = 'D'
    elif(num == 0):
        l = 'NG'
    return l    

In [59]:
# create grade sheet of student
def gradesheet(last_digit):
    sheet = marksheet(last_digit)
    # rescale
    rsheet = sheet.applymap(re_scale)
    gpsheet = rsheet.applymap(conv_into_gp)
    gpsheet = gpsheet.T
    gpsheet.columns=['GRADE POINT']
    conditions = [
        (gpsheet['GRADE POINT'] == 4.0),
        (gpsheet['GRADE POINT'] == 3.6),
        (gpsheet['GRADE POINT'] == 3.2),
        (gpsheet['GRADE POINT'] == 2.8),
        (gpsheet['GRADE POINT'] == 2.4),
        (gpsheet['GRADE POINT'] == 2.0),
        (gpsheet['GRADE POINT'] == 1.6),
        (gpsheet['GRADE POINT'] == 0),
    ]
    choices = ['A+','A','B+','B','C+','C','D', 'NG']
    gpsheet['GRADE'] = np.select(conditions, choices)
    return gpsheet

In [60]:
# create full grade_sheet except final grade column
def full_grade_sheet(last_digit):
    temp = gradesheet(last_digit)
    temp = pd.concat([sub_data, temp], axis=1)
    temp = temp.dropna(axis = 0)
    return temp

In [61]:
# create final grade column
def final_grade_col(last_digit):
    # select total sub marks
    temp = student(last_digit)
    temp = temp.iloc[:,4:] # remove details other than marks
    temp = temp[["en","ne","so","ec","bu","ac","ch","ed","on","oh"]]
    temp = temp.dropna(axis = 1) # drop unselected and subjects that has no marks
    temp = temp.applymap(conv_into_gp)
    temp = temp.applymap(gp_to_letter)
    temp = temp.T
    return temp

In [65]:
fgs = full_grade_sheet
si = student_info
fgs(5).to_excel("foo.xlsx")